In [47]:
import pandas as pd
import os
import json
from tqdm.auto import tqdm
import networkx as nx
import semanticscholar as sch
import urllib
import json
import requests
from fuzzywuzzy import fuzz
import statistics
from networkx.algorithms.dag import descendants

In [ ]:
paper_attributes = pd.read_csv("paper_attributes.csv")
our_paper_attributes =  paper_attributes[paper_attributes['Attribute'].isin(['AA.AfId','AA.AfN','AA.AuId','AA.AuN','AA.S','AW','BT','BV','C.CId','C.CN','CC','CitCon','D','ECC','F.FId','F.FN','Id','Pt','RId','Ti','Y'])]
our_paper_attributes

In [20]:
cur_path = os.getcwd()

file_path_iclr = os.path.relpath('..\\data\\paper_data_mag\\MAG_ICLR.json', cur_path)
file_path_acl = os.path.relpath('..\\data\\paper_data_mag\\MAG_ACL.json', cur_path)
file_path_cvpr = os.path.relpath('..\\data\\paper_data_mag\\MAG_CVPR.json', cur_path)
file_path_emnlp = os.path.relpath('..\\data\\paper_data_mag\\MAG_EMNLP.json', cur_path)
file_path_iccv = os.path.relpath('..\\data\\paper_data_mag\\MAG_ICCV.json', cur_path)
file_path_iclr = os.path.relpath('..\\data\\paper_data_mag\\MAG_ICLR.json', cur_path)
file_path_icml = os.path.relpath('..\\data\\paper_data_mag\\MAG_ICML.json', cur_path)
file_path_naacl = os.path.relpath('..\\data\\paper_data_mag\\MAG_NAACL.json', cur_path)
file_path_neurips = os.path.relpath('..\\data\\paper_data_mag\\MAG_NEURIPS.json', cur_path)

In [59]:
def create_paper_data_df(file_path):
    with open(file_path) as f_in:
        paper_data_dict = json.load(f_in)
    paper_data_records = paper_data_dict['entities']
    paper_data_df = pd.DataFrame.from_records(paper_data_records)
    return paper_data_df

def fuzzy_matching(x,y):
    return True if fuzz.ratio(x,y) > 90 else False

In [22]:
iclr_paper_data = create_paper_data_df(file_path_iclr)
references_list_iclr = [(rid,row['Id']) for idx,row in tqdm(iclr_paper_data.iterrows()) if type(row['RId'])!=float for rid in row['RId']]

acl_paper_data = create_paper_data_df(file_path_acl)
references_list_acl = [(rid,row['Id']) for idx,row in tqdm(acl_paper_data.iterrows()) if type(row['RId'])!=float for rid in row['RId']]

cvpr_paper_data = create_paper_data_df(file_path_cvpr)
references_list_cvpr = [(rid,row['Id']) for idx,row in tqdm(cvpr_paper_data.iterrows()) if type(row['RId'])!=float for rid in row['RId']]

emnlp_paper_data = create_paper_data_df(file_path_emnlp)
references_list_emnlp = [(rid,row['Id']) for idx,row in tqdm(emnlp_paper_data.iterrows()) if type(row['RId'])!=float for rid in row['RId']]

iccv_paper_data = create_paper_data_df(file_path_iccv)
references_list_iccv = [(rid,row['Id']) for idx,row in tqdm(iccv_paper_data.iterrows()) if type(row['RId'])!=float for rid in row['RId']]

icml_paper_data = create_paper_data_df(file_path_icml)
references_list_icml = [(rid,row['Id']) for idx,row in tqdm(icml_paper_data.iterrows()) if type(row['RId'])!=float for rid in row['RId']]

naacl_paper_data = create_paper_data_df(file_path_naacl)
references_list_naacl = [(rid,row['Id']) for idx,row in tqdm(naacl_paper_data.iterrows()) if type(row['RId'])!=float for rid in row['RId']]

neurips_paper_data = create_paper_data_df(file_path_neurips)
references_list_neurips = [(rid,row['Id']) for idx,row in tqdm(neurips_paper_data.iterrows()) if type(row['RId'])!=float for rid in row['RId']]


1735it [00:00, 8015.37it/s]
2594it [00:00, 4784.73it/s]
5236it [00:00, 5980.71it/s]
1817it [00:00, 5829.72it/s]
1701it [00:00, 6090.51it/s]
3485it [00:00, 11973.94it/s]
1390it [00:00, 6935.02it/s]
2516it [00:00, 11451.22it/s]


In [23]:
all_paper_data_df = pd.concat([iclr_paper_data, acl_paper_data, cvpr_paper_data, emnlp_paper_data, iccv_paper_data, icml_paper_data, naacl_paper_data, neurips_paper_data])
all_paper_data_df.to_pickle("..\\data\\all_paper_data.pkl")

In [24]:
all_paper_data_df = pd.read_pickle("../data/all_paper_data.pkl")
all_paper_data_df.set_index('Id')

,logprob,prob,Ti,AW,Pt,Y,D,CC,ECC,DN,BV,BT,AA,F,C,RId,CitCon,DOI,FP
Id,,,,,,,,,,,,,,,,,,,
2962835968,-12.778,2.822183e-06,very deep convolutional networks for large sca...,"[work, investigate, effect, convolutional, net...",3,2015,2015-01-01,11290,20451,Very Deep Convolutional Networks for Large-Sca...,ICLR 2015 : International Conference on Learni...,p,"[{'AuN': 'karen simonyan', 'AuId': 2033942250,...","[{'FN': 'convolution', 'FId': 152003226}, {'FN...","{'CN': 'iclr', 'CId': 2584161585}",NaN,NaN,NaN,NaN
2964121744,-13.079,2.088635e-06,adam a method for stochastic optimization,"[introduce, adam, algorithm, first, order, gra...",3,2015,2015-01-01,15036,19483,Adam: A Method for Stochastic Optimization,ICLR 2015 : International Conference on Learni...,p,"[{'AuN': 'diederik p kingma', 'AuId': 20556046...","[{'FN': 'stochastic optimization', 'FId': 1943...","{'CN': 'iclr', 'CId': 2584161585}",NaN,NaN,NaN,NaN
2964308564,-13.715,1.105735e-06,neural machine translation by jointly learning...,"[neural, machine, translation, recently, appro...",3,2015,2015-01-01,9174,11050,Neural Machine Translation by Jointly Learning...,ICLR 2015 : International Conference on Learni...,p,"[{'AuN': 'dzmitry bahdanau', 'AuId': 250910180...","[{'FN': 'machine translation', 'FId': 20300521...","{'CN': 'iclr', 'CId': 2584161585}","[2064675550, 2157331557, 3037950864, 213233900...",{'194249466': ['Neural machine translation is ...,NaN,NaN
2963542991,-14.389,5.635555e-07,overfeat integrated recognition localization a...,"[integrated, framework, convolutional, network...",3,2014,2014-02-23,2142,2934,"OverFeat: Integrated Recognition, Localization...",ICLR 2014 : International Conference on Learni...,p,"[{'AuN': 'pierre sermanet', 'AuId': 37905947, ...","[{'FN': 'deep learning', 'FId': 108583219}, {'...","{'CN': 'iclr', 'CId': 2584161585}",NaN,NaN,NaN,NaN
1959608418,-14.390,5.629922e-07,auto encoding variational bayes,"[perform, efficient, inference, learning, dire...",3,2014,2014-01-01,3369,4291,Auto-Encoding Variational Bayes,ICLR 2014 : International Conference on Learni...,p,"[{'AuN': 'diederik p kingma', 'AuId': 20556046...","[{'FN': 'approximate inference', 'FId': 277747...","{'CN': 'iclr', 'CId': 2584161585}","[2146502635, 2163922914, 2145094598, 216685163...",NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2285793032,-22.804,1.248385e-10,hypernode graphs for learning from binary rela...,"[aim, paper, propose, methods, learning, inter...",3,2015,2015-01-29,0,0,Hypernode Graphs for Learning from Binary Rela...,"Networks: From Graphs to Rich Data, NIPS Works...",p,"[{'AuN': 'thomas ricatte', 'AuId': 256014509, ...","[{'FN': 'chordal graph', 'FId': 160446614}, {'...","{'CN': 'neurips', 'CId': 1127325140}","[2153975459, 2165922980, 108936587, 2005676288...",NaN,NaN,NaN
2483983853,-22.846,1.197039e-10,applying machine learning methods to improve s...,NaN,3,2014,2014-12-01,0,0,Applying Machine Learning Methods to Improve S...,Neural Information Processing Systems,p,"[{'AuN': 'bases team', 'AuId': 2809558853, 'Af...","[{'FN': 'medical research', 'FId': 106977388},...","{'CN': 'neurips', 'CId': 1127325140}",NaN,NaN,NaN,NaN
2259661006,-22.941,1.088555e-10,a newton type incremental method with a superl...,NaN,3,2015,2015-01-01,0,0,A Newton-type Incremental Method with a Superl...,Proceedings of NIPS Workshop on Optimization f...,p,"[{'AuN': 'r anton', 'AuId': 2271936098, 'AfId'...","[{'FN': 'rate of convergence', 'FId': 57869625...","{'CN': 'neurips', 'CId': 1127325140}",NaN,NaN,NaN,NaN


In [25]:
aw = all_paper_data_df.set_index('Id')['AW']
f = all_paper_data_df.set_index('Id')['F']

In [25]:
reference_df_iclr = pd.DataFrame(references_list_iclr, columns=['this_paper_infected','got_infected_by'])
reference_df_acl = pd.DataFrame(references_list_acl, columns=['this_paper_infected','got_infected_by'])
reference_df_cvpr = pd.DataFrame(references_list_cvpr, columns=['this_paper_infected','got_infected_by'])
reference_df_emnlp = pd.DataFrame(references_list_emnlp, columns=['this_paper_infected','got_infected_by'])
reference_df_iccv = pd.DataFrame(references_list_iccv, columns=['this_paper_infected','got_infected_by'])
reference_df_icml = pd.DataFrame(references_list_icml, columns=['this_paper_infected','got_infected_by'])
reference_df_naacl = pd.DataFrame(references_list_naacl, columns=['this_paper_infected','got_infected_by'])
reference_df_neurips = pd.DataFrame(references_list_neurips, columns=['this_paper_infected','got_infected_by'])

reference_df = pd.concat([reference_df_iclr, reference_df_acl, reference_df_cvpr, reference_df_emnlp, reference_df_iccv, reference_df_icml, reference_df_naacl, reference_df_neurips])
reference_df.to_pickle("..\\data\\networks\\citation_network.pkl")

In [14]:
reference_df = pd.read_pickle('../data/networks/citation_network.pkl')
reference_df

,this_paper_infected,got_infected_by
0,2963403868,2785994986
1,2963207607,2785994986
2,2134557905,2785994986
3,2154579312,2785994986
4,2966661,2785994986
...,...,...
2191,2482888308,2897127218
2192,2057624533,2897127218
2193,2103458172,2897127218
2194,2136885855,2897127218


In [72]:
valid_idx_infection = []
all_paper_ids = set(all_paper_data_df['Id'].values)
aw = all_paper_data_df.set_index('Id')['AW']
f = all_paper_data_df.set_index('Id')['F']
# print(all_paper_ids)
for idx,row in tqdm(reference_df.iterrows()):
    
    source = row['this_paper_infected']
    destination = row['got_infected_by']
    if source in all_paper_ids and destination in all_paper_ids:
        source_abstract = aw.at[source]
        dest_abstract = aw.at[destination]
        source_fos = f.at[source]
        dest_fos = f.at[destination]
        
        # Make sure these fields aren't NaNs
        if type(source_abstract) != float and type(dest_abstract) != float and type(source_fos) != float and type(dest_fos) != float:
            #abstract overlap
            source_abstract = set(source_abstract)
            dest_abstract = set(dest_abstract)
            abstract_overlap_score = len(set.intersection(source_abstract,dest_abstract))/min(len(source_abstract), len(dest_abstract))
            #FOS overlap
            source_fos = set([pair['FN'] for pair in source_fos])
            dest_fos = set([pair['FN'] for pair in dest_fos])
            fos_overlap_score = len(set.intersection(source_fos,dest_fos))/min(len(source_fos), len(dest_fos))
            if abstract_overlap_score > 0.1 and fos_overlap_score > 0.1:
                valid_idx_infection.append(idx)


In [73]:
infected_df = reference_df.iloc[valid_idx_infection, :]
# infected_df.to_pickle("..\\data\\networks\\citation_infection_network.pkl")
# infected_df = pd.read_pickle("../data/networks/citation_infection_network.pkl")
infected_df

,this_paper_infected,got_infected_by
6,2576915720,2785994986
7,2136026194,2785994986
8,2340427832,2785994986
13,2963403868,2996428491
14,2250539671,2996428491
...,...,...
2143,2963108696,2995358843
2151,2155027007,2996031640
2158,2950492145,2996031640
2166,1533861849,2995492258


In [52]:
cur_path = os.getcwd()
review_df = pd.read_pickle("../data/paper_quality/review_data.pkl")

review_df['mag_id'] = [None]*len(review_df)
for idx,row in tqdm(all_paper_data_df.iterrows()):
    if row['Y'] != 2018:
        continue
    if row['C']['CN'] != 'iclr':
        continue
    match_series = review_df['title'].apply(lambda x : fuzzy_matching(x,row['Ti']))
    match = review_df.loc[match_series == True]
    if len(match)>0:
        review_df.loc[match.index[0],'mag_id'] = row['Id']

In [60]:
review_df_with_mag_id = review_df.query('mag_id > 0')

In [11]:
# review_df.to_pickle("..\\data\\paper_quality\\review_data_with_mag_id.pkl")
# review_df_with_mag_id = pd.read_pickle("..\data\paper_quality\review_data_with_mag_id.pkl")
# review_df_with_mag_id = pd.read_pickle("../data/paper_quality/review_data_with_mag_id.pkl")
all_paper_data_df = pd.read_pickle("../data/all_paper_data.pkl")

In [74]:
edge_endpoints = set(infected_df.values.flatten())
print(len(edge_endpoints))

36578


In [75]:
patient_zero_ids = [row['mag_id'] for idx,row in review_df_with_mag_id.iterrows() if row['mag_id'] is not None]
patient_zero_ratings = [review_df_with_mag_id[review_df_with_mag_id['mag_id'] == mag_id]['rating'].values[0] for mag_id in patient_zero_ids]

none_counter = 0
counter = 0
affiliation_df = pd.read_pickle("../data/prestige/prestige_data.pkl")
patient_zero_prestiges = []
for patient_zero in patient_zero_ids:
    author_affiliations = [authdict['AfId'] for authdict in all_paper_data_df[all_paper_data_df['Id'] == patient_zero]['AA'].values[0]]
    prestiges = [affiliation_df[affiliation_df['id'] == affiliation]['prestige'].values[0] for affiliation in author_affiliations if affiliation is not None]
    prestiges = [item for item in prestiges if str(item) != 'nan']
    if len(prestiges)>0:
        patient_zero_prestiges.append(statistics.mean(prestiges))
        counter += 1
    else:
        patient_zero_prestiges.append(None)
        none_counter += 1
print(counter)
print(none_counter)

patient_zero_df = pd.DataFrame()
patient_zero_df['mag_id'] = patient_zero_ids
patient_zero_df['rating'] = patient_zero_ratings
patient_zero_df['prestige'] = patient_zero_prestiges
patient_zero_df = patient_zero_df[~patient_zero_df['prestige'].isnull()]
patient_zero_df = patient_zero_df[patient_zero_df['mag_id'].map(lambda x: x in edge_endpoints)]
patient_zero_df

39
52


,mag_id,rating,prestige
1,2963001136,good,0.521330
5,2963477238,good,0.248911
7,2786760026,good,0.878647
8,2963940260,good,0.321361
10,2786916331,good,0.290745
14,2963197656,good,0.364832
19,2963626025,good,0.917072
23,2963803379,good,0.917072
25,2771430488,good,0.917072
31,2964190622,good,0.303238


In [76]:
G_infection = nx.from_pandas_edgelist(infected_df, source='this_paper_infected', target='got_infected_by',create_using=nx.DiGraph())

In [78]:
patient_zero_df['descendants'] = patient_zero_df.loc[~patient_zero_df['prestige'].isnull(),'mag_id'].map(lambda x : descendants(G_infection,x))
patient_zero_df['n_descendants'] = patient_zero_df['descendants'].map(len)
patient_zero_df.shape

(19, 5)

In [44]:
patient_zero_df[~patient_zero_df['prestige'].isnull()]

0     False
1     False
2     False
3      True
4     False
      ...  
72    False
73    False
74     True
75    False
76     True
Name: prestige, Length: 77, dtype: bool

In [115]:
descendants(G_infection,2787849668)

NetworkXError: The node 2787849668 is not in the graph.